# ОИАД. Лабораторная работа №3
Для построения моделей: datasets/insurance_train.csv Для оценки обобщающей способности: datasets/insurance_test.csv

Инфо о датасете: https://www.kaggle.com/datasets/mosapabdelghany/medical-insurance-cost-dataset

### Задание 1. Подготовка данных
- проверить наличие пропусков и выбросов
- привести категориальные признаки к числовым
- вычислить парные корреляции признаков

In [3]:
# 1. ПОДГОТОВКА ДАТАСЕТА
import pandas as pd
import numpy as np

# Загрузка данных
df = pd.read_csv('insurance.csv')

# Проверка пропусков
print("Пропуски в данных:", df.isnull().sum().sum())

# Проверка выбросов по правилу 1.5 * IQR
print("\nПроверка выбросов:")
numeric_cols = ['age', 'bmi', 'children', 'charges']
for col in numeric_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
    print(f"{col}: {len(outliers)} выбросов")

# Кодирование категориальных признаков
df_encoded = pd.get_dummies(df, columns=['sex', 'smoker', 'region'], drop_first=True)

# Парные корреляции
correlation_matrix = df_encoded.corr()
print("\nКорреляция с charges:")
print(correlation_matrix['charges'].sort_values(ascending=False))

Пропуски в данных: 0

Проверка выбросов:
age: 0 выбросов
bmi: 9 выбросов
children: 0 выбросов
charges: 139 выбросов

Корреляция с charges:
charges             1.000000
smoker_yes          0.787251
age                 0.299008
bmi                 0.198341
region_southeast    0.073982
children            0.067998
sex_male            0.057292
region_northwest   -0.039905
region_southwest   -0.043210
Name: charges, dtype: float64


Поиск выбросов - используется правило 1.5 * IQR (межквартильный размах):
Q1 - 25-й процентиль, Q3 - 75-й процентиль
IQR = Q3 - Q1 (размах средних 50% данных)
Выбросы - значения за пределами [Q1 - 1.5*IQR, Q3 + 1.5*IQR]

Результаты: Пропусков не обнаружено, выбросы идентифицированы методом IQR (Interquartile Range). Наибольшая корреляция с целевой переменной наблюдается у признака smoker и age.

### Задание 2. Многомерная линейная регрессия
Построить модель линейной регрессии и подобрать параметры:
- аналитически (реализовать самому)
- численно, с помощью методов градиентного спуска (реализовать самому)

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

# Подготовка данных
X = df_encoded.drop('charges', axis=1)
y = df_encoded['charges']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

X_train_scaled = np.column_stack([np.ones(X_train_scaled.shape[0]), X_train_scaled])
X_test_scaled = np.column_stack([np.ones(X_test_scaled.shape[0]), X_test_scaled])

print("Размеры данных:")
print(f"X_train: {X_train_scaled.shape}, y_train: {y_train.shape}")
print(f"X_test: {X_test_scaled.shape}, y_test: {y_test.shape}")

# 2.а Аналитическое решение (нормальное уравнение)
def analytical_solution(X, y):
    theta = np.linalg.inv(X.T @ X) @ X.T @ y
    return theta

theta_analytical = analytical_solution(X_train_scaled, y_train)
y_pred_analytical = X_test_scaled @ theta_analytical

mse_analytical = mean_squared_error(y_test, y_pred_analytical)
r2_analytical = r2_score(y_test, y_pred_analytical)

print("\nАНАЛИТИЧЕСКОЕ РЕШЕНИЕ:")
print(f"Коэффициенты: {theta_analytical}")
print(f"MSE: {mse_analytical:.2f}")
print(f"R²: {r2_analytical:.4f}")

# 2.б Градиентный спуск
def gradient_descent(X, y, learning_rate=0.01, n_iter=1000):
    m = len(y)
    theta = np.zeros(X.shape[1]) 
    cost_history = []
    
    for i in range(n_iter):
        y_pred = X @ theta
        gradient = (1/m) * X.T @ (y_pred - y)
        theta -= learning_rate * gradient
        
        cost = (1/(2*m)) * np.sum((y_pred - y)**2)
        cost_history.append(cost)
                
        # if i % 200 == 0:
        #     print(f"Iteration {i}: Cost = {cost:.2f}")
    return theta, cost_history

theta_gd, cost_history = gradient_descent(X_train_scaled, y_train, learning_rate=0.01, n_iter=1000)

y_pred_gd = X_test_scaled @ theta_gd

mse_gd = mean_squared_error(y_test, y_pred_gd)
r2_gd = r2_score(y_test, y_pred_gd)

print("\nГРАДИЕНТНЫЙ СПУСК:")
print(f"Коэффициенты: {theta_gd}")
print(f"MSE: {mse_gd:.2f}")
print(f"R²: {r2_gd:.4f}")

# Сравнение методов
print("\nСРАВНЕНИЕ МЕТОДОВ:")
print(f"Аналитическое решение - MSE: {mse_analytical:.2f}, R²: {r2_analytical:.4f}")
print(f"Градиентный спуск - MSE: {mse_gd:.2f}, R²: {r2_gd:.4f}")
print(f"Разница в R²: {abs(r2_analytical - r2_gd):.6f}")

# Получим названия признаков после кодирования
feature_names = ['intercept'] + list(X.columns)
print("\nКоэффициенты аналитического решения:")
for i, coef in enumerate(theta_analytical):
    print(f"{feature_names[i]}: {coef:+.2f}")

Размеры данных:
X_train: (1070, 9), y_train: (1070,)
X_test: (268, 9), y_test: (268,)

АНАЛИТИЧЕСКОЕ РЕШЕНИЕ:
Коэффициенты: [ 1.33460897e+04  3.61497541e+03  2.03622812e+03  5.16890247e+02
 -9.29310107e+00  9.55848141e+03 -1.58140981e+02 -2.90157047e+02
 -3.49110678e+02]
MSE: 33596915.85
R²: 0.7836

ГРАДИЕНТНЫЙ СПУСК:
Коэффициенты: [ 1.33455136e+04  3.61488185e+03  2.03241312e+03  5.16980755e+02
 -8.70529765e+00  9.55789194e+03 -1.43936208e+02 -2.74066666e+02
 -3.34265195e+02]
MSE: 33608763.19
R²: 0.7835

СРАВНЕНИЕ МЕТОДОВ:
Аналитическое решение - MSE: 33596915.85, R²: 0.7836
Градиентный спуск - MSE: 33608763.19, R²: 0.7835
Разница в R²: 0.000076

Коэффициенты аналитического решения:
intercept: +13346.09
age: +3614.98
bmi: +2036.23
children: +516.89
sex_male: -9.29
smoker_yes: +9558.48
region_northwest: -158.14
region_southeast: -290.16
region_southwest: -349.11


Многомерная линейная регрессия моделирует зависимость целевой переменной как линейную комбинацию признаков: y = w₀ + w₁x₁ + ... + wₙxₙ. 
Аналитическое решение использует нормальное уравнение (XᵀX)⁻¹Xᵀy, которое находит оптимальные веса за одну операцию. 
Градиентный спуск итеративно обновляет веса в направлении антиградиента функции потерь.

Результаты: 
Оба метода показали схожие результаты (MSE ~33 млн). 
Небольшая разница обусловлена численной погрешностью: аналитическое решение точнее, но требует обращения матрицы, что может быть вычислительно сложно для больших датасетов. 
Градиентный спуск более масштабируем, но требует подбора темпа обучения.

В целом, можно сделать выводы, что:
Курение увеличивает стоимость страховки на ~$9,558 - самый значительный фактор
Каждый год возраста добавляет ~$3,615 к стоимости
Модель успешно предсказывает 78% variations в стоимости страховки

intercept: +13346.09
smoker_yes: +9558.48
age: +3614.98
bmi: +2036.23
children: +516.89

### Задание 3. Добавление регуляризации
Модифицировать линейную модель путем добавления регуляризационного слагаемого. Найти оптимальные веса:
- аналитически
- численно

In [ ]:
import numpy as np
from sklearn.metrics import r2_score

def ridge_analytical(X, y, alpha=1.0):
    m, n = X.shape
    I = np.eye(n)
    I[0, 0] = 0  
    return np.linalg.inv(X.T @ X + alpha * I) @ X.T @ y

# Подбор alpha
alphas = [0.001, 0.01, 0.1, 1, 10, 100]
best_alpha = alphas[0]
best_r2 = -np.inf

# Перебираем разные alpha и выбираем лучший по R² на тестовой выборке
for alpha in alphas:
    theta_ridge = ridge_analytical(X_train_scaled, y_train, alpha)
    r2 = r2_score(y_test, X_test_scaled @ theta_ridge)
    if r2 > best_r2:
        best_r2, best_alpha = r2, alpha

# Обучение с лучшим alpha
theta_ridge_optimal = ridge_analytical(X_train_scaled, y_train, best_alpha)
y_pred_ridge = X_test_scaled @ theta_ridge_optimal

print("RIDGE РЕГРЕССИЯ (аналитически):")
print(f"Лучший alpha: {best_alpha}, R²: {r2_score(y_test, y_pred_ridge):.4f}")

# 3.б Градиентный спуск с регуляризацией
def ridge_gd(X, y, alpha=1.0, lr=0.01, n_iter=1000):
    theta = np.zeros(X.shape[1])
    for _ in range(n_iter):
        y_pred = X @ theta
        gradient = (1/len(y)) * X.T @ (y_pred - y) + (alpha/len(y)) * theta
        gradient[0] = (1/len(y)) * X[:, 0] @ (y_pred - y)
        theta -= lr * gradient
    return theta

theta_ridge_gd = ridge_gd(X_train_scaled, y_train, best_alpha)
y_pred_ridge_gd = X_test_scaled @ theta_ridge_gd

print("\nRIDGE РЕГРЕССИЯ (градиентный спуск):")
print(f"R²: {r2_score(y_test, y_pred_ridge_gd):.4f}")

# Сравнение
print("\nСРАВНЕНИЕ:")
print(f"Обычная: R² = {r2_analytical:.4f}")
print(f"Ridge аналит: R² = {r2_score(y_test, y_pred_ridge):.4f}")
print(f"Ridge GD: R² = {r2_score(y_test, y_pred_ridge_gd):.4f}")

RIDGE РЕГРЕССИЯ (аналитически):
Лучший alpha: 0.001, R²: 0.7836

RIDGE РЕГРЕССИЯ (градиентный спуск):
R²: 0.7835

СРАВНЕНИЕ:
Обычная: R² = 0.7836
Ridge аналит: R² = 0.7836
Ridge GD: R² = 0.7835


Регуляризация добавляет штраф за большие значения весов к функции потерь. 
Ridge регрессия добавляет штраф за большие коэффициенты
Alpha - параметр регуляризации (чем больше, тем сильнее штраф), alpha=0.001 очень маленькое значение. Это означает, что регуляризация почти не применяется. 
Данные хорошо обусловлены, нет сильной мультиколлинеарности.
Intercept (свободный член) - это базовое значение целевой переменной, когда все признаки равны 0. По сути это начальная стоимость страховки при "нулевых" условиях. Его мы не регуляризуем, потому как
    Intercept - это просто смещение всей модели
    Его регуляризация не имеет смысла, так как он не связан с каким-либо признаком
    Мы хотим штрафовать только коэффициенты при признаках, чтобы бороться с переобучением

Аналитическое решение Ridge регрессии
    Формула: theta = (X^T * X + alpha * I)^(-1) * X^T * y
    I - единичная матрица, но intercept не регуляризуем (I[0,0] = 0)

Градиентный спуск с L2-регуляризацией
    Градиент: (1/m)*X^T*(X*theta - y) + (alpha/m)*theta
    Intercept (theta[0]) не регуляризуем

Результаты: 
    Все модели показывают одинаково высокое качество - R² ≈ 0.784
    Градиентный спуск сошелся к правильному решению - разница с аналитическим методом всего 0.0001
    Реализация корректна - оба метода дают практически идентичные результаты

### Задание 4. Оценка обобщающей способности
Сравнить между собой модели на тестовых данных по среднему квадрату ошибки:
- константную - прогноз средним значением
- из пункта 2
- из пункта 3

In [14]:
y_pred_constant = np.full_like(y_test, np.mean(y_train))

models = {
    "Константная": y_pred_constant,
    "Линейная (аналит)": y_pred_analytical,
    "Линейная (GD)": y_pred_gd,
    "Ridge (аналит)": y_pred_ridge,
    "Ridge (GD)": y_pred_ridge_gd
}

print("СРАВНЕНИЕ МОДЕЛЕЙ НА ТЕСТЕ:")
print(f"{'Модель':<20} {'MSE':<12} {'R²':<8}")
for name, y_pred in models.items():
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    print(f"{name:<20} {mse:<12.2f} {r2:<8.4f}")

СРАВНЕНИЕ МОДЕЛЕЙ НА ТЕСТЕ:
Модель               MSE          R²      
Константная          155391443.68 -0.0009 
Линейная (аналит)    33596915.85  0.7836  
Линейная (GD)        33608763.19  0.7835  
Ridge (аналит)       33596923.81  0.7836  
Ridge (GD)           33608771.05  0.7835  


Обобщающая способность - способность модели хорошо работать на новых, ранее не виденных данных. 
Константная модель (прогноз средним) служит baseline для сравнения. 
MSE (Mean Squared Error) измеряет средний квадрат ошибок и является стандартной метрикой для задач регрессии.

Результаты: 
    Все регрессионные модели значительно превосходят константную и показывают хорошие и почти одинаковые результаты.
    Лучшими являются Линейная (аналитическая) и иRidge (аналитическая).